# Data Extractions from OpenEO

To run the extractions, you need an account in the [Copernicus Data Space Ecosystem (CDSE)](https://dataspace.copernicus.eu/).

In [ ]:
!pip install git+https://github.com/WorldCereal/prometheo.git@scaleag_augmentations
!pip install git+https://github.com/ScaleAGData/scaleag-vito.git@prometheo-integration

In [ ]:
from loguru import logger
import geopandas as gpd
from pathlib import Path
from scaleagdata_vito.openeo.extract_sample_scaleag import (
    generate_input_for_extractions,
    extract
)
from scaleagdata_vito.presto.presto_df import load_dataset

### Assess data correctness before launching the OpenEO jobs 
You can run some checks on your input file to make sure they are suitable to run the extractions successfully. In particular, it is important to check the validity of the geometries and, ideally, to also have a column containing a unique id for each sample.

In case of invalid geometries, you will be provided with both the dataframe with the failing polygons to be fixed and the one with valid geometries.


In [ ]:
def check_unique_id(df_path, unique_id):
    df = gpd.read_file(df_path)
    if df[unique_id].nunique() != df.shape[0]:
        logger.info("IDs are not unique!")
        return df[df[unique_id].duplicated(keep=False)]
    else:
        logger.info("IDs are unique")
        return None

def check_valid_geometry(df):
    if isinstance(df, str):
        df = gpd.read_file(df)
    df_invalid = df[~df.geometry.is_valid]
    # Assessing wheather some invalid geometries are present
    if len(df_invalid) > 0:
        # 1) some invalid geometries are present. Attempt fixing them
        df['geometry'] = df.geometry.buffer(0)
        df_invalid = df[~df.geometry.is_valid]
        df_valid = df[df.geometry.is_valid]
        if len(df_invalid) > 0:
            # 2) Still some invalid geometries are present. Return them
            logger.info("Invalid geometries found! Returning invalid geometries")
            return df_invalid, df_valid
        else:
            # All geometries are now valid. Return fixed dataframe and empty dataframe for invalid geometries
            logger.info("Fixed some invalid geometries. All geometries are now valid")
            return gpd.GeoDataFrame(), df
    else:
        # All geometries are valid. Return empty dataframe for invalid geometries
        logger.info("All geometries are valid")
        return gpd.GeoDataFrame(), df

def _save(save_to, original_file_path, df, valid=False):
    if valid:
        filename = Path(save_to) / f"{Path(original_file_path).stem}_valid.geojson"
    else:
        filename = Path(save_to) / f"{Path(original_file_path).stem}_invalid.geojson"
    logger.info(f"Saving invalid geometries to {filename}")
    Path(save_to).mkdir(parents=True, exist_ok=True)
    df.to_file(filename)

In [ ]:
input_file = "/projects/TAP/HEScaleAgData/data/GEOMaize/corrected/.geojson"
invalid_geom, valid_geom = check_valid_geometry(input_file)
non_unique_ids = check_unique_id(input_file, unique_id="Field_ID")

In [ ]:
# save files after geometry validity check. If invalid geometries are present, save them to a separate file
if len(invalid_geom) > 0:
    _save(
        save_to="/projects/TAP/HEScaleAgData/data/GEOMaize/",
        original_file_path=input_file,
        df=invalid_geom,
        valid=False,
    )

# save valid geometries to a separate file
_save(
    save_to="/projects/TAP/HEScaleAgData/data/GEOMaize/",
    original_file_path=input_file,
    df=valid_geom,
    valid=True,
)

#### Provide job instructions and start OpenEO extractions

1) Indicate the following fields in order to guide the extraction
2) In the cell below you will be asked for authentication and be provided with a link. click on the link and login with your CDSE credentials.  
3) Once the extraction process will be over, you will find your extracted dataset in the output folder you indicated. You can load it by running `load_dataset` as shown below

    ```python
    job_params = dict(
        output_folder=..., # where to save the extracted dataset
        input_df=..., # input georeferenced dataset to run the extractions for 
        start_date=..., # string indicating from which date to extract data  
        end_date=..., # string indicating until which date to extract the data 
        unique_id_column=..., # name of the column in the input_df containing the unique ID of the samples  
        composite_window=..., # "month" or "dekad" are supported. Default is "dekad"
    )

    ```

In [ ]:
job_params = dict(
    output_folder="/home/giorgia/Private/data/geomaize/Maize_yield_2021/",
    input_df="/home/giorgia/Private/data/geomaize/Maize_2021.geojson",
    start_date="2021-07-01",
    end_date="2021-10-31",
    unique_id_column="Field_ID",
    composite_window="dekad",
)
extract(generate_input_for_extractions(job_params))

In [ ]:
dataset_df = load_dataset(job_params["output_folder"])